# **Urban Infrastructure Stress Analysis:**

Geospatial Data Analysis – HSLU Master’s Program


## Mount Google Drive in Collab

In [10]:
# from google.colab import drive
from pathlib import Path

# 1. Mount Google Drive
# drive.mount('/content/drive')

# 2. Base path for the project (EDIT THIS ONLY)
BASE = Path(".").resolve()  # start from the folder where the notebook is running

# 3. Data folders
DATA = BASE / "Data"
RAW = DATA / "raw"
PROC = DATA / "processed"

print("Project base folder:", BASE)
print("Raw data folder:", RAW)
print("Processed data folder:", PROC)

# Check if raw folder exists
if RAW.exists():
    print("\nFiles in RAW folder:")
    for p in RAW.iterdir():
        print(" -", p.name)
else:
    print("\n❌ ERROR: RAW folder not found. Please check your BASE path.")


Project base folder: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project
Raw data folder: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/raw
Processed data folder: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/processed

Files in RAW folder:
 - str_stadtquartier_raw_for_join.csv
 - traffic_data_cleaned.csv
 - population_raw.csv


## Importing the necessary libaries

In [11]:
# =========================================================
# 0. IMPORTS
# =========================================================

# --- Core Python Libraries ---
import pandas as pd
import numpy as np

# --- File & Path Handling ---
from pathlib import Path

# --- Geospatial Libraries (added later if needed) ---
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer

# --- Visualization Libraries (if needed in Colab) ---
# import matplotlib.pyplot as plt
# import seaborn as sns

# --- Machine Learning / Clustering (added later) ---
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# --- Utility ---
import warnings
warnings.filterwarnings("ignore")

print("Imports loaded successfully.")


Imports loaded successfully.


## Loading and inspecting the raw data

In [12]:
# =========================================================
# 2. LOAD AND INSPECT RAW DATA
# =========================================================

import pandas as pd

# 2.1 Define file paths (using RAW from Section 1)
traffic_path = RAW / "traffic_data_cleaned.csv"
pop_path     = RAW / "population_raw.csv"
quart_path   = RAW / "str_stadtquartier_raw_for_join.csv"

print("Traffic file path :", traffic_path)
print("Population path   :", pop_path)
print("Quartier path     :", quart_path)

# 2.2 Traffic data (sample only, file is very large)
print("\n=== TRAFFIC DATA (sample) ===")
print("Approx. file size (GB):", round(traffic_path.stat().st_size / 1e9, 3))

traffic_sample = pd.read_csv(traffic_path, nrows=100_000)  # sample for inspection
print("Sample shape:", traffic_sample.shape)
display(traffic_sample.head())
print("\nTraffic columns:\n", traffic_sample.columns.tolist())

# 2.3 Population data (load full)
print("\n=== POPULATION DATA ===")
population = pd.read_csv(pop_path)
print("Population shape:", population.shape)
display(population.head())
print("\nPopulation columns:\n", population.columns.tolist())

# 2.4 Quartier lookup table
print("\n=== QUARTIER LOOKUP TABLE ===")
quartiers = pd.read_csv(quart_path)
print("Quartier table shape:", quartiers.shape)
display(quartiers.head())
print("\nQuartier columns:\n", quartiers.columns.tolist())


Traffic file path : /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/raw/traffic_data_cleaned.csv
Population path   : /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/raw/population_raw.csv
Quartier path     : /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/raw/str_stadtquartier_raw_for_join.csv

=== TRAFFIC DATA (sample) ===
Approx. file size (GB): 3.735
Sample shape: (100000, 14)


,measurement_site_id,measurement_site_name,counting_site_id,counting_site_name,position_description,east_coordinate,north_coordinate,direction,signal_id,signal_name,num_detectors,timestamp,vehicle_count,vehicle_count_status
0,Z001M001,Unknown,Z001,Seestrasse (Strandbad Wollishofen),Unknown,2683009.89,1243936.2,outbound,789,Badanstalt Wollishofen,1,2013-01-01T00:00:00,224.0,Measured
1,Z001M001,Unknown,Z001,Seestrasse (Strandbad Wollishofen),Unknown,2683009.89,1243936.2,outbound,789,Badanstalt Wollishofen,1,2013-01-01T01:00:00,547.0,Measured
2,Z001M001,Unknown,Z001,Seestrasse (Strandbad Wollishofen),Unknown,2683009.89,1243936.2,outbound,789,Badanstalt Wollishofen,1,2013-01-01T02:00:00,249.0,Measured
3,Z001M001,Unknown,Z001,Seestrasse (Strandbad Wollishofen),Unknown,2683009.89,1243936.2,outbound,789,Badanstalt Wollishofen,1,2013-01-01T03:00:00,123.0,Measured
4,Z001M001,Unknown,Z001,Seestrasse (Strandbad Wollishofen),Unknown,2683009.89,1243936.2,outbound,789,Badanstalt Wollishofen,1,2013-01-01T04:00:00,95.0,Measured



Traffic columns:
 ['measurement_site_id', 'measurement_site_name', 'counting_site_id', 'counting_site_name', 'position_description', 'east_coordinate', 'north_coordinate', 'direction', 'signal_id', 'signal_name', 'num_detectors', 'timestamp', 'vehicle_count', 'vehicle_count_status']

=== POPULATION DATA ===
Population shape: (225557, 18)


,StichtagDatJahr,StichtagDatMM,StichtagDatMonat,StichtagDat,SexCd,SexLang,AlterV20ueber80Sort_noDM,AlterV20ueber80Cd_noDM,AlterV20ueber80Kurz_noDM,HerkunftCd,HerkunftLang,KreisCd,KreisLang,QuarCd,QuarLang,DatenstandCd,DatenstandLang,AnzBestWir
0,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,11,Rathaus,V,Veröffentlicht,114
1,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,12,Hochschulen,V,Veröffentlicht,24
2,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,13,Lindenhof,V,Veröffentlicht,37
3,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,14,City,V,Veröffentlicht,48
4,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,2,Kreis 2,21,Wollishofen,V,Veröffentlicht,950



Population columns:
 ['StichtagDatJahr', 'StichtagDatMM', 'StichtagDatMonat', 'StichtagDat', 'SexCd', 'SexLang', 'AlterV20ueber80Sort_noDM', 'AlterV20ueber80Cd_noDM', 'AlterV20ueber80Kurz_noDM', 'HerkunftCd', 'HerkunftLang', 'KreisCd', 'KreisLang', 'QuarCd', 'QuarLang', 'DatenstandCd', 'DatenstandLang', 'AnzBestWir']

=== QUARTIER LOOKUP TABLE ===
Quartier table shape: (56162, 12)


,adresse,anzahl_fla_projektiert,anzahl_fla_real,flaeche_projektiert,flaeche_real,flaeche_total,gwr_egid,hausnummer,lokalisationsname,objectid,stadtkreis,statistisches_quartier
0,Zähringerstrasse 43,NaN,2.0,NaN,449.0,449,140003,43,Zähringerstrasse,1,1.0,Rathaus
1,Limmatquai 144,NaN,1.0,NaN,443.0,443,140004,144,Limmatquai,2,1.0,Rathaus
2,Limmatquai 142,NaN,1.0,NaN,141.0,141,140005,142,Limmatquai,3,1.0,Rathaus
3,Niederdorfstrasse 76,NaN,1.0,NaN,188.0,188,140007,76,Niederdorfstrasse,4,1.0,Rathaus
4,Niederdorfstrasse 80,NaN,2.0,NaN,107.0,107,140008,80,Niederdorfstrasse,5,1.0,Rathaus



Quartier columns:
 ['adresse', 'anzahl_fla_projektiert', 'anzahl_fla_real', 'flaeche_projektiert', 'flaeche_real', 'flaeche_total', 'gwr_egid', 'hausnummer', 'lokalisationsname', 'objectid', 'stadtkreis', 'statistisches_quartier']


### Traffic Data

In [13]:
# =========================================================
# 3A. TRAFFIC DATA PREPARATION
# =========================================================


# Use only the columns we need
traffic_cols = [
    "measurement_site_id",
    "east_coordinate",
    "north_coordinate",
    "timestamp",
    "vehicle_count"
]

traffic_path = RAW / "traffic_data_cleaned.csv"

# Load only the needed columns
traffic = pd.read_csv(
    traffic_path,
    usecols=traffic_cols,
    parse_dates=["timestamp"]  # auto-parse datetime
)

print("Traffic loaded.")
traffic.head()


Traffic loaded.


,measurement_site_id,east_coordinate,north_coordinate,timestamp,vehicle_count
0,Z001M001,2683009.89,1243936.2,2013-01-01 00:00:00,224.0
1,Z001M001,2683009.89,1243936.2,2013-01-01 01:00:00,547.0
2,Z001M001,2683009.89,1243936.2,2013-01-01 02:00:00,249.0
3,Z001M001,2683009.89,1243936.2,2013-01-01 03:00:00,123.0
4,Z001M001,2683009.89,1243936.2,2013-01-01 04:00:00,95.0


| Column                  | Why we need it                                                     |
| ----------------------- | ------------------------------------------------------------------ |
| **measurement_site_id** | Groups all records from the same station; ensures unique locations |
| **east_coordinate**     | Needed to map station location into Zurich quarters                |
| **north_coordinate**    | Needed to map station location into Zurich quarters                                                     |
| **timestamp**           | Extract year (and later day/hour patterns if needed)               |
| **vehicle_count**       | The actual traffic value                                           |


Traffic Data Quality Checks

In [14]:
# =========================================================
# 3A.4 TRAFFIC DATA QUALITY CHECKS
# =========================================================

print("Checking missing values...\n")
missing = traffic.isna().sum()
print(missing)

print("\nChecking duplicates...\n")
duplicate_rows = traffic.duplicated().sum()
print("Total duplicate rows:", duplicate_rows)

print("\nChecking unique measurement sites...\n")
print("Unique measurement_site_id:", traffic["measurement_site_id"].nunique())

print("\nChecking timestamp range...\n")
print("Min timestamp:", traffic["timestamp"].min())
print("Max timestamp:", traffic["timestamp"].max())

print("\nChecking coordinate sanity...\n")
print("East min/max:", traffic["east_coordinate"].min(), traffic["east_coordinate"].max())
print("North min/max:", traffic["north_coordinate"].min(), traffic["north_coordinate"].max())


Checking missing values...

measurement_site_id          0
east_coordinate              0
north_coordinate             0
timestamp                    0
vehicle_count          1369306
dtype: int64

Checking duplicates...

Total duplicate rows: 194

Checking unique measurement sites...

Unique measurement_site_id: 219

Checking timestamp range...

Min timestamp: 2012-01-01 00:00:00
Max timestamp: 2025-11-12 23:00:00

Checking coordinate sanity...

East min/max: 2678017.34 2687630.91
North min/max: 1242969.91 1253804.18


Cleaning and saving a new file

In [15]:
# =========================================================
# 3A.5 CLEAN TRAFFIC DATA (remove missing & duplicates)
# =========================================================

traffic_clean = traffic.copy()

# 1. Remove rows with missing vehicle_count
before_missing = traffic_clean.shape[0]
traffic_clean = traffic_clean.dropna(subset=["vehicle_count"])
after_missing = traffic_clean.shape[0]

print(f"Removed {before_missing - after_missing} rows with missing vehicle_count.")

# 2. Remove duplicate rows
before_dupes = traffic_clean.shape[0]
traffic_clean = traffic_clean.drop_duplicates()
after_dupes = traffic_clean.shape[0]

print(f"Removed {before_dupes - after_dupes} duplicate rows.")

# 3. Reset index
traffic_clean = traffic_clean.reset_index(drop=True)

traffic_clean.info()


Removed 1369306 rows with missing vehicle_count.
Removed 38 duplicate rows.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20352149 entries, 0 to 20352148
Data columns (total 5 columns):
 #   Column               Dtype         
---  ------               -----         
 0   measurement_site_id  object        
 1   east_coordinate      float64       
 2   north_coordinate     float64       
 3   timestamp            datetime64[ns]
 4   vehicle_count        float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 776.4+ MB


In [16]:
# =========================================================
# 3A.6 CONVERT COORDINATES FROM LV95 TO WGS84
# =========================================================

# Create a transformer from Swiss LV95 (EPSG:2056) to WGS84 (EPSG:4326)
# LV95 uses East/North, WGS84 uses Lat/Lon
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

print("Converting coordinates from LV95 (EPSG:2056) to WGS84 (EPSG:4326)...")

# Transform coordinates
# transformer.transform expects (x, y) which is (east, north) for LV95
# and returns (longitude, latitude) for WGS84
traffic_clean["longitude"], traffic_clean["latitude"] = transformer.transform(
    traffic_clean["east_coordinate"].values,
    traffic_clean["north_coordinate"].values
)

print("Conversion complete!")
print("\nSample of converted coordinates:")
display(traffic_clean[["measurement_site_id", "east_coordinate", "north_coordinate",
                       "longitude", "latitude"]].head(10))

# Quick sanity check: Zurich coordinates should be around:
# Latitude: ~47.37 (North)
# Longitude: ~8.54 (East)
print("\nCoordinate ranges (should be in Zurich area):")
print(f"Latitude range: {traffic_clean['latitude'].min():.4f} to {traffic_clean['latitude'].max():.4f}")
print(f"Longitude range: {traffic_clean['longitude'].min():.4f} to {traffic_clean['longitude'].max():.4f}")

Converting coordinates from LV95 (EPSG:2056) to WGS84 (EPSG:4326)...
Conversion complete!

Sample of converted coordinates:


,measurement_site_id,east_coordinate,north_coordinate,longitude,latitude
0,Z001M001,2683009.89,1243936.2,8.537068,47.341059
1,Z001M001,2683009.89,1243936.2,8.537068,47.341059
2,Z001M001,2683009.89,1243936.2,8.537068,47.341059
3,Z001M001,2683009.89,1243936.2,8.537068,47.341059
4,Z001M001,2683009.89,1243936.2,8.537068,47.341059
5,Z001M001,2683009.89,1243936.2,8.537068,47.341059
6,Z001M001,2683009.89,1243936.2,8.537068,47.341059
7,Z001M001,2683009.89,1243936.2,8.537068,47.341059
8,Z001M001,2683009.89,1243936.2,8.537068,47.341059
9,Z001M001,2683009.89,1243936.2,8.537068,47.341059



Coordinate ranges (should be in Zurich area):
Latitude range: 47.3325 to 47.4297
Longitude range: 8.4720 to 8.5987


In [17]:
clean_path = PROC / "traffic_clean_small.csv"
traffic_clean.to_csv(clean_path, index=False)
print("Saved cleaned file to:", clean_path)


Saved cleaned file to: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/processed/traffic_clean_small.csv


### Population Data

In [18]:
# 3B.1 Load population dataset

pop_path = RAW / "population_raw.csv"
print("Population file path:", pop_path)

population = pd.read_csv(pop_path)

population.head()
population.columns.tolist()



Population file path: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/raw/population_raw.csv


['StichtagDatJahr',
 'StichtagDatMM',
 'StichtagDatMonat',
 'StichtagDat',
 'SexCd',
 'SexLang',
 'AlterV20ueber80Sort_noDM',
 'AlterV20ueber80Cd_noDM',
 'AlterV20ueber80Kurz_noDM',
 'HerkunftCd',
 'HerkunftLang',
 'KreisCd',
 'KreisLang',
 'QuarCd',
 'QuarLang',
 'DatenstandCd',
 'DatenstandLang',
 'AnzBestWir']

Loding only the needed Column

In [19]:
# Look at how categories are coded in the original dataset
population["SexCd"].unique(), population["HerkunftCd"].unique(), population["AlterV20ueber80Cd_noDM"].unique()


(array([  1,   2, 252]), array([1, 2]), array([1, 2, 3, 4, 5]))

In [20]:
population_jan = population[population["StichtagDatMM"] == 1]
population_jan.head()


,StichtagDatJahr,StichtagDatMM,StichtagDatMonat,StichtagDat,SexCd,SexLang,AlterV20ueber80Sort_noDM,AlterV20ueber80Cd_noDM,AlterV20ueber80Kurz_noDM,HerkunftCd,HerkunftLang,KreisCd,KreisLang,QuarCd,QuarLang,DatenstandCd,DatenstandLang,AnzBestWir
0,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,11,Rathaus,V,Veröffentlicht,114
1,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,12,Hochschulen,V,Veröffentlicht,24
2,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,13,Lindenhof,V,Veröffentlicht,37
3,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,14,City,V,Veröffentlicht,48
4,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,2,Kreis 2,21,Wollishofen,V,Veröffentlicht,950


In [21]:
# ============================================================
# Inspect Demographic Categories in the Population Dataset
# This helps us understand how the dataset is split into
# demographic subgroups (sex, origin, age).
# We will use this to correctly aggregate total population.
# ============================================================

# Check how sex is coded (male/female)
print("SEX CODES (SexCd):")
print(population_jan["SexCd"].value_counts())
print("\n1 = male, 2 = female")

# Check how origin (Swiss/Foreign) is coded
print("\nORIGIN CODES (HerkunftCd):")
print(population_jan["HerkunftCd"].value_counts())
print("\n1 = Swiss, 2 = Foreign")

# Check age group codes
print("\nAGE GROUP CODES (AlterV20ueber80Cd_noDM):")
print(population_jan["AlterV20ueber80Cd_noDM"].value_counts())
print("\nAge groups 1–5 represent different age bands (e.g. 0–19, 20–39, etc.)")



SEX CODES (SexCd):
SexCd
2    9521
1    9511
Name: count, dtype: int64

1 = male, 2 = female

ORIGIN CODES (HerkunftCd):
HerkunftCd
1    9543
2    9489
Name: count, dtype: int64

1 = Swiss, 2 = Foreign

AGE GROUP CODES (AlterV20ueber80Cd_noDM):
AlterV20ueber80Cd_noDM
3    3820
4    3819
2    3818
1    3814
5    3761
Name: count, dtype: int64

Age groups 1–5 represent different age bands (e.g. 0–19, 20–39, etc.)


NOTE:
Each quarter-year contains multiple demographic rows:
- 2 sexes × 2 origins × 5 age groups = 20 rows per quarter-year.
To obtain TRUE population, we must sum over all demographic categories

In [22]:
# ============================================================
# 3B.4 Aggregate TRUE population per year & quarter (January only)
# We now:
# 1) Keep only the columns we need for aggregation
# 2) Group by Year + Quarter + Quarter name
# 3) Sum AnzBestWir over ALL demographic categories
#    (both sexes, both origins, all age groups)
#    -> this gives the TRUE population for each quarter-year.
# ============================================================

# 1) Keep only the relevant columns from the January snapshot
population_jan_small = population_jan[[
    "StichtagDatJahr",   # year
    "QuarCd",            # quarter ID (numeric)
    "QuarLang",          # quarter name (string)
    "AnzBestWir"         # population count for each demographic slice
]]

# 2) Aggregate to total population per (year, quarter)
population_year_quarter = (
    population_jan_small
    .groupby(["StichtagDatJahr", "QuarCd", "QuarLang"], as_index=False)
    .agg(total_population=("AnzBestWir", "sum"))
)

# Quick look at the result
population_year_quarter.head()


,StichtagDatJahr,QuarCd,QuarLang,total_population
0,1998,11,Rathaus,3098
1,1998,12,Hochschulen,773
2,1998,13,Lindenhof,1025
3,1998,14,City,999
4,1998,21,Wollishofen,15831


In [23]:
population_year_quarter.groupby("StichtagDatJahr")["total_population"].sum()


StichtagDatJahr
1998    359411
1999    359989
2000    360960
2001    361641
2002    362744
2003    364791
2004    364828
2005    365695
2006    367745
2007    371140
2008    377715
2009    381080
2010    383114
2011    386091
2012    389828
2013    394154
2014    398433
2015    404959
2016    410168
2017    416461
2018    422390
2019    428837
2020    433757
2021    434050
2022    435975
2023    442748
2024    446547
2025    448512
Name: total_population, dtype: int64

Sanity Check

In [24]:
# ============================================================
# 3B.7 Check for Missing Values and Duplicates in Final Dataset
# This is important before saving the clean dataset to disk.
#
# We check:
# - Missing values in any column
# - Total number of duplicated rows
# - Dataset size (rows, columns)
# ============================================================

print("=== CHECK MISSING VALUES ===\n")
print(population_year_quarter.isna().sum())

print("\n=== CHECK DUPLICATED ROWS ===\n")
dup_count = population_year_quarter.duplicated().sum()
print("Number of duplicated rows:", dup_count)

print("\n=== DATASET SHAPE ===")
print("Rows:", population_year_quarter.shape[0])
print("Columns:", population_year_quarter.shape[1])


=== CHECK MISSING VALUES ===

StichtagDatJahr     0
QuarCd              0
QuarLang            0
total_population    0
dtype: int64

=== CHECK DUPLICATED ROWS ===

Number of duplicated rows: 0

=== DATASET SHAPE ===
Rows: 958
Columns: 4


In [25]:
# ============================================================
# 3Save Cleaned Population Dataset
#
# This file will be used in:
#  - Tableau visualizations
#  - Spatial join with Zurich quarters
#  - Merging with traffic data
#  - Stress Index calculation
# ============================================================

save_path = PROC / "population_year_quarter_clean.csv"
population_year_quarter.to_csv(save_path, index=False)

print("Clean population dataset saved to:")
print(save_path)


Clean population dataset saved to:
/Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/processed/population_year_quarter_clean.csv


### Quartier Data

In [26]:
# ============================================================
# 3C.1 Load Quartier Lookup Dataset
# This dataset maps addresses / GWR IDs to:
#  - statistisches_quartier (quarter)
#  - stadtkreis (district)
#  - address information
# ============================================================

quartier_path = RAW / "str_stadtquartier_raw_for_join.csv"

quartiers = pd.read_csv(quartier_path)

print("Quartier dataset loaded.")
print("Shape:", quartiers.shape)
quartiers.head()


Quartier dataset loaded.
Shape: (56162, 12)


,adresse,anzahl_fla_projektiert,anzahl_fla_real,flaeche_projektiert,flaeche_real,flaeche_total,gwr_egid,hausnummer,lokalisationsname,objectid,stadtkreis,statistisches_quartier
0,Zähringerstrasse 43,NaN,2.0,NaN,449.0,449,140003,43,Zähringerstrasse,1,1.0,Rathaus
1,Limmatquai 144,NaN,1.0,NaN,443.0,443,140004,144,Limmatquai,2,1.0,Rathaus
2,Limmatquai 142,NaN,1.0,NaN,141.0,141,140005,142,Limmatquai,3,1.0,Rathaus
3,Niederdorfstrasse 76,NaN,1.0,NaN,188.0,188,140007,76,Niederdorfstrasse,4,1.0,Rathaus
4,Niederdorfstrasse 80,NaN,2.0,NaN,107.0,107,140008,80,Niederdorfstrasse,5,1.0,Rathaus


Selecting Relevant Columns

In [27]:
# ================================================
# 3C.2 Keep Only Relevant Quartier Columns
#  - gwr_egid: building ID (sometimes useful for merges)
#  - adresse: optional for labels
#  - stadtkreis: district ID
#  - statistisches_quartier: quarter name
# ================================================

quartier_clean = quartiers[[
    "gwr_egid",
    "adresse",
    "stadtkreis",
    "statistisches_quartier"
]].copy()

quartier_clean.head()


,gwr_egid,adresse,stadtkreis,statistisches_quartier
0,140003,Zähringerstrasse 43,1.0,Rathaus
1,140004,Limmatquai 144,1.0,Rathaus
2,140005,Limmatquai 142,1.0,Rathaus
3,140007,Niederdorfstrasse 76,1.0,Rathaus
4,140008,Niederdorfstrasse 80,1.0,Rathaus


Sanity Check

In [28]:
# ============================================================
# 3C.3 Sanity Check for Quartier Lookup (clean version)
# Checks:
#   - Missing values per column
#   - Number of duplicated rows
#   - Final dataset shape
# ============================================================

print("=== SANITY CHECK: Quartier Lookup Dataset ===\n")

# 1) Missing values per column
missing = quartier_clean.isna().sum()
print(">> Missing values per column:\n")
print(missing)
print("\n")

# 2) Duplicate rows
duplicates = quartier_clean.duplicated().sum()
print(">> Number of duplicated rows:", duplicates)
print("\n")

# 3) Dataset shape
rows, cols = quartier_clean.shape
print(">> Dataset shape:")
print(f"Rows: {rows}")
print(f"Columns: {cols}")

print("\n=== Sanity Check Completed ===")


=== SANITY CHECK: Quartier Lookup Dataset ===

>> Missing values per column:

gwr_egid                  0
adresse                   2
stadtkreis                2
statistisches_quartier    2
dtype: int64


>> Number of duplicated rows: 0


>> Dataset shape:
Rows: 56162
Columns: 4

=== Sanity Check Completed ===


In [29]:
# ============================================================
# 3C.4 Save Clean Quartier Dataset
# This makes the dataset ready for merging and visualization
# ============================================================

# Define save path relative to the project root
SAVE_DIR = PROC
SAVE_DIR.mkdir(parents=True, exist_ok=True)
quartier_save_path = SAVE_DIR / "quartier_clean.csv"

# Save the dataframe
quartier_clean.to_csv(quartier_save_path, index=False)

print("Quartier dataset saved successfully at:")
print(quartier_save_path)


Quartier dataset saved successfully at:
/Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/processed/quartier_clean.csv


# Use the same RAW folder defined at the top
print("Raw data folder:", RAW)
for item in RAW.iterdir():
    print(" -", item.name)


In [30]:
# Use the same RAW folder defined at the top
print("Raw data folder:", RAW)
for item in RAW.iterdir():
    print(" -", item.name)


Raw data folder: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/raw
 - str_stadtquartier_raw_for_join.csv
 - traffic_data_cleaned.csv
 - population_raw.csv


In [34]:
QUARTER_DIR = RAW / "Stadtkreise_und_Quartiere_Zurich_und_Winterthur_-OGD/"

# List all files in that folder
!ls "$QUARTER_DIR"


Produktblatt_Stadtkreise_und_Quartiere_Zurich_und_Winterthur_-OGD.pdf
UP_GEMEINDEN_STADTKREISE_F.cpg
UP_GEMEINDEN_STADTKREISE_F.dbf
UP_GEMEINDEN_STADTKREISE_F.prj
UP_GEMEINDEN_STADTKREISE_F.shp
UP_GEMEINDEN_STADTKREISE_F.shx
UP_GEMEINDEN_STADTQUARTIERE_F.cpg
UP_GEMEINDEN_STADTQUARTIERE_F.dbf
UP_GEMEINDEN_STADTQUARTIERE_F.prj
UP_GEMEINDEN_STADTQUARTIERE_F.shp
UP_GEMEINDEN_STADTQUARTIERE_F.shx
UP_STADTKREISE_F.cpg
UP_STADTKREISE_F.dbf
UP_STADTKREISE_F.prj
UP_STADTKREISE_F.shp
UP_STADTKREISE_F.shx
UP_STADTQUARTIERE_F.cpg
UP_STADTQUARTIERE_F.dbf
UP_STADTQUARTIERE_F.prj
UP_STADTQUARTIERE_F.shp
UP_STADTQUARTIERE_F.shx


In [35]:
import geopandas as gpd

quartier_shp = QUARTER_DIR / "UP_STADTQUARTIERE_F.shp"

quarters = gpd.read_file(quartier_shp)

print("Loaded quarter shapefile!")
print("Shape:", quarters.shape)
print(quarters.columns)

quarters.head()


Loaded quarter shapefile!
Shape: (34, 7)
Index(['BFS', 'GEMEINDENA', 'QUARTIERNA', 'QUARTIERNU', 'SHAPE_AREA',
       'SHAPE_LEN', 'geometry'],
      dtype='object')


,BFS,GEMEINDENA,QUARTIERNA,QUARTIERNU,SHAPE_AREA,SHAPE_LEN,geometry
0,261,Zürich,City,14,5.950569e+05,5323.712005,"POLYGON ((2683110.53 1248572.193, 2683168.166 ..."
1,261,Zürich,Langstrasse,42,1.211799e+06,5930.518592,"POLYGON ((2681465.865 1248839.333, 2681465.123..."
2,261,Zürich,Gewerbeschule,51,7.302896e+05,4803.636786,"POLYGON ((2682227.502 1249306.497, 2682226.608..."
3,261,Zürich,Hottingen,72,5.052667e+06,15029.355436,"POLYGON ((2686575.849 1249472.507, 2686578.541..."
4,261,Zürich,Hard,44,1.391492e+06,6587.429715,"POLYGON ((2680312.08 1249441.429, 2680350.87 1..."


Quarter Mapping

In [36]:
import geopandas as gpd

# You already have this:
# quarters = gpd.read_file(quartier_shp)

# Keep only Zurich (not Winterthur)
quarters_zurich = quarters[quarters["GEMEINDENA"] == "Zürich"].copy()
print("Zurich quarters:", quarters_zurich.shape)

# Ensure CRS is LV95
print("Quarters CRS before:", quarters_zurich.crs)
if quarters_zurich.crs is None:
    quarters_zurich = quarters_zurich.set_crs(epsg=2056)


Zurich quarters: (34, 7)
Quarters CRS before: EPSG:2056


### Loading the Clean Version

In [37]:
# ==========================================================
# LOAD ALL CLEANED DATASETS FROM /processed FOLDER
# ==========================================================


SAVE_DIR = PROC 

# traffic_clean_path = BASE + "traffic_clean_small.csv"
# population_clean_path = BASE + "population_year_quarter_clean.csv"
# quartier_clean_path = BASE + "quartier_clean.csv"
traffic_clean_path = PROC / "traffic_clean_small.csv"
population_clean_path = PROC / "population_year_quarter_clean.csv"
quartier_clean_path = PROC / "quartier_clean.csv"

# Load them
traffic_clean = pd.read_csv(traffic_clean_path)
population_clean = pd.read_csv(population_clean_path)
quartier_clean = pd.read_csv(quartier_clean_path)

print("Datasets loaded successfully.\n")

print("Traffic shape:", traffic_clean.shape)
print("Population shape:", population_clean.shape)
print("Quartier shape:", quartier_clean.shape)

print("=== TRAFFIC CLEAN (HEAD) ===")
display(traffic_clean.head())

print("\n=== POPULATION CLEAN (HEAD) ===")
display(population_clean.head())

print("\n=== QUARTIER CLEAN (HEAD) ===")
display(quartier_clean.head())


Datasets loaded successfully.

Traffic shape: (20352149, 7)
Population shape: (958, 4)
Quartier shape: (56162, 4)
=== TRAFFIC CLEAN (HEAD) ===


,measurement_site_id,east_coordinate,north_coordinate,timestamp,vehicle_count,longitude,latitude
0,Z001M001,2683009.89,1243936.2,2013-01-01 00:00:00,224.0,8.537068,47.341059
1,Z001M001,2683009.89,1243936.2,2013-01-01 01:00:00,547.0,8.537068,47.341059
2,Z001M001,2683009.89,1243936.2,2013-01-01 02:00:00,249.0,8.537068,47.341059
3,Z001M001,2683009.89,1243936.2,2013-01-01 03:00:00,123.0,8.537068,47.341059
4,Z001M001,2683009.89,1243936.2,2013-01-01 04:00:00,95.0,8.537068,47.341059



=== POPULATION CLEAN (HEAD) ===


,StichtagDatJahr,QuarCd,QuarLang,total_population
0,1998,11,Rathaus,3098
1,1998,12,Hochschulen,773
2,1998,13,Lindenhof,1025
3,1998,14,City,999
4,1998,21,Wollishofen,15831



=== QUARTIER CLEAN (HEAD) ===


,gwr_egid,adresse,stadtkreis,statistisches_quartier
0,140003,Zähringerstrasse 43,1.0,Rathaus
1,140004,Limmatquai 144,1.0,Rathaus
2,140005,Limmatquai 142,1.0,Rathaus
3,140007,Niederdorfstrasse 76,1.0,Rathaus
4,140008,Niederdorfstrasse 80,1.0,Rathaus


## Merging Dataset

Quartier Mapping

### Quarter Mapping via Spatial Join

Goal: Assign each traffic measurement site to a statistical quarter so that we can later aggregate
traffic volumes per quarter and year.

Inputs:
- `traffic` (cleaned traffic data with `measurement_site_id`, `east_coordinate`, `north_coordinate`)
- `quarters_zurich` (GeoDataFrame with quarter polygons from `UP_STADTQUARTIERE_F.shp`)

Method:
1. Extract all **unique measurement sites** and their coordinates.
2. Convert the sites to a GeoDataFrame in the Swiss coordinate system LV95 (EPSG:2056).

1.   List item
2.   List item


3. Use a **spatial join** (`gpd.sjoin`) between the sites and the quarter polygons.
4. This assigns each site to a quarter name (`QUARTIERNA`) and quarter number (`QUARTIERNR`).

Output:
- `sites_with_quarter`: GeoDataFrame with one row per measurement site and its assigned quarter.


In [39]:

print("=== Traffic Columns ===")
print(traffic.columns)
print("\nTraffic preview:")
display(traffic.head())

print("\n=== Quartier_clean Columns ===")
print(quartier_clean.columns)
print("\nQuartier preview:")
display(quartier_clean.head())

print("\n=== Population Columns ===")
print(population.columns)
print("\nPopulation preview:")
display(population.head())


=== Traffic Columns ===
Index(['measurement_site_id', 'east_coordinate', 'north_coordinate',
       'timestamp', 'vehicle_count'],
      dtype='object')

Traffic preview:


,measurement_site_id,east_coordinate,north_coordinate,timestamp,vehicle_count
0,Z001M001,2683009.89,1243936.2,2013-01-01 00:00:00,224.0
1,Z001M001,2683009.89,1243936.2,2013-01-01 01:00:00,547.0
2,Z001M001,2683009.89,1243936.2,2013-01-01 02:00:00,249.0
3,Z001M001,2683009.89,1243936.2,2013-01-01 03:00:00,123.0
4,Z001M001,2683009.89,1243936.2,2013-01-01 04:00:00,95.0



=== Quartier_clean Columns ===
Index(['gwr_egid', 'adresse', 'stadtkreis', 'statistisches_quartier'], dtype='object')

Quartier preview:


,gwr_egid,adresse,stadtkreis,statistisches_quartier
0,140003,Zähringerstrasse 43,1.0,Rathaus
1,140004,Limmatquai 144,1.0,Rathaus
2,140005,Limmatquai 142,1.0,Rathaus
3,140007,Niederdorfstrasse 76,1.0,Rathaus
4,140008,Niederdorfstrasse 80,1.0,Rathaus



=== Population Columns ===
Index(['StichtagDatJahr', 'StichtagDatMM', 'StichtagDatMonat', 'StichtagDat',
       'SexCd', 'SexLang', 'AlterV20ueber80Sort_noDM',
       'AlterV20ueber80Cd_noDM', 'AlterV20ueber80Kurz_noDM', 'HerkunftCd',
       'HerkunftLang', 'KreisCd', 'KreisLang', 'QuarCd', 'QuarLang',
       'DatenstandCd', 'DatenstandLang', 'AnzBestWir'],
      dtype='object')

Population preview:


,StichtagDatJahr,StichtagDatMM,StichtagDatMonat,StichtagDat,SexCd,SexLang,AlterV20ueber80Sort_noDM,AlterV20ueber80Cd_noDM,AlterV20ueber80Kurz_noDM,HerkunftCd,HerkunftLang,KreisCd,KreisLang,QuarCd,QuarLang,DatenstandCd,DatenstandLang,AnzBestWir
0,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,11,Rathaus,V,Veröffentlicht,114
1,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,12,Hochschulen,V,Veröffentlicht,24
2,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,13,Lindenhof,V,Veröffentlicht,37
3,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,1,Kreis 1,14,City,V,Veröffentlicht,48
4,1998,1,Januar,1998-01-31,1,männlich,1,1,0-19,1,Schweizer*in,2,Kreis 2,21,Wollishofen,V,Veröffentlicht,950


In [41]:
# Keep only needed columns from Zurich quarters
quarters_slim = quarters_zurich[["QUARTIERNA", "QUARTIERNU", "geometry"]]

import geopandas as gpd

# 1. get one row per measurement site with LV95 coordinates
sites = (
    traffic_clean[["measurement_site_id", "east_coordinate", "north_coordinate"]]
    .dropna(subset=["east_coordinate", "north_coordinate"])
    .drop_duplicates(subset="measurement_site_id")
)

# 2. convert to a GeoDataFrame in EPSG:2056 (same CRS as the quarters)
gdf_sites = gpd.GeoDataFrame(
    sites,
    geometry=gpd.points_from_xy(sites["east_coordinate"], sites["north_coordinate"]),
    crs="EPSG:2056"
)

# 3. proceed with the spatial join
quarters_slim = quarters_zurich[["QUARTIERNA", "QUARTIERNU", "geometry"]]
sites_with_quarter = gpd.sjoin(gdf_sites, quarters_slim, how="left", predicate="within")
# Spatial join: assign each site to a quarter
sites_with_quarter = gpd.sjoin(
    gdf_sites,
    quarters_slim,
    how="left",
    predicate="within"
)

print("Sites with quarter info:", sites_with_quarter.shape)
sites_with_quarter.head()


Sites with quarter info: (219, 7)


,measurement_site_id,east_coordinate,north_coordinate,geometry,index_right,QUARTIERNA,QUARTIERNU
0,Z001M001,2683009.89,1243936.20,POINT (2683009.89 1243936.2),19,Wollishofen,21
8756,Z001M002,2683013.18,1243937.64,POINT (2683013.18 1243937.64),19,Wollishofen,21
17512,Z002M001,2682480.51,1243162.50,POINT (2682480.51 1243162.5),19,Wollishofen,21
26265,Z002M002,2682484.91,1243162.41,POINT (2682484.91 1243162.41),19,Wollishofen,21
35021,Z003M001,2681573.79,1242969.91,POINT (2681573.79 1242969.91),18,Leimbach,23


In [42]:
# Create a clean site → quarter mapping and merge into full traffic data

# 5.1 Make a clean mapping table (drop geometry + index from spatial join)
sites_quarter_map = (
    sites_with_quarter
    .drop(columns=["geometry", "index_right"])
    .rename(columns={"QUARTIERNA": "quarter_name",
                     "QUARTIERNU": "quarter_id"})
)

print("Site → quarter mapping:")
display(sites_quarter_map.head())

# 5.2 Merge the mapping back into the full traffic dataset
traffic_q = traffic.merge(
    sites_quarter_map[["measurement_site_id", "quarter_name", "quarter_id"]],
    on="measurement_site_id",
    how="left"
)

print("Traffic with quarter info:", traffic_q.shape)
display(traffic_q.head())


Site → quarter mapping:


,measurement_site_id,east_coordinate,north_coordinate,quarter_name,quarter_id
0,Z001M001,2683009.89,1243936.20,Wollishofen,21
8756,Z001M002,2683013.18,1243937.64,Wollishofen,21
17512,Z002M001,2682480.51,1243162.50,Wollishofen,21
26265,Z002M002,2682484.91,1243162.41,Wollishofen,21
35021,Z003M001,2681573.79,1242969.91,Leimbach,23


Traffic with quarter info: (21721493, 7)


,measurement_site_id,east_coordinate,north_coordinate,timestamp,vehicle_count,quarter_name,quarter_id
0,Z001M001,2683009.89,1243936.2,2013-01-01 00:00:00,224.0,Wollishofen,21
1,Z001M001,2683009.89,1243936.2,2013-01-01 01:00:00,547.0,Wollishofen,21
2,Z001M001,2683009.89,1243936.2,2013-01-01 02:00:00,249.0,Wollishofen,21
3,Z001M001,2683009.89,1243936.2,2013-01-01 03:00:00,123.0,Wollishofen,21
4,Z001M001,2683009.89,1243936.2,2013-01-01 04:00:00,95.0,Wollishofen,21


Saving The Qauartier Data with the Quarter Information

In [46]:

#  Save the enriched traffic dataset with quarter information
output_path = PROC / "traffic_with_quarters.csv"

traffic_q.to_csv(output_path, index=False)

print("Saved enriched traffic dataset to:", output_path)


Saved enriched traffic dataset to: /Users/dongyuangao/Desktop/geospatial_analysis_hslu_project/Data/processed/traffic_with_quarters.csv


## Aggreate Yearly Traffic per Quarter

 **Aggregate Yearly Traffic per Quarter**

Goal: Compute the total annual traffic volume for each statistical quarter.

Input:
- `traffic_q`: enriched traffic dataset  
  (one row per measurement event with `vehicle_count`, `timestamp`, `quarter_name`, `quarter_id`)

Method:
1. Convert `timestamp` to a datetime object and extract the calendar year.
2. Group by `quarter_name`, `quarter_id`, and `year`.
3. Sum `vehicle_count` within each group to obtain `total_traffic`.

Output:
- `traffic_qy`: table with one row per (quarter, year) and the corresponding `total_traffic`.
- This table will later be merged with the population dataset to compute growth rates and the Stress Index.


In [47]:
# Aggregate yearly traffic per quarter

import pandas as pd

# 7.1 Ensure timestamp is datetime and create a 'year' column
traffic_q["timestamp"] = pd.to_datetime(traffic_q["timestamp"], errors="coerce")

# Drop rows where timestamp could not be parsed (if any)
before_drop = traffic_q.shape[0]
traffic_q = traffic_q.dropna(subset=["timestamp"])
after_drop = traffic_q.shape[0]

print(f"Dropped {before_drop - after_drop} rows with invalid timestamps.")

traffic_q["year"] = traffic_q["timestamp"].dt.year

# 7.2 Aggregate total yearly traffic per quarter
traffic_qy = (
    traffic_q
    .groupby(["quarter_name", "quarter_id", "year"], as_index=False)["vehicle_count"]
    .sum()
    .rename(columns={"vehicle_count": "total_traffic"})
    .sort_values(["quarter_id", "year"])
)

print("Traffic per quarter & year:", traffic_qy.shape)
display(traffic_qy.head(10))


Dropped 0 rows with invalid timestamps.
Traffic per quarter & year: (415, 4)


,quarter_name,quarter_id,year,total_traffic
247,Rathaus,11,2012,12528744.0
248,Rathaus,11,2013,12979295.0
249,Rathaus,11,2014,12482229.0
250,Rathaus,11,2015,7759976.0
251,Rathaus,11,2016,12228606.0
252,Rathaus,11,2017,12034869.0
253,Rathaus,11,2018,11743323.0
254,Rathaus,11,2019,11749113.0
255,Rathaus,11,2020,11147403.0
256,Rathaus,11,2021,11429011.0


## Merge Yearly Traffic with Yearly Population Data

### Merge Yearly Traffic with Yearly Population Data

Goal:
Combine the aggregated annual traffic per quarter (`traffic_qy`) with the annual population
per quarter (`population`) so that we can compute growth rates and the Stress Index.

Matching keys:
- Quarter name: `traffic_qy.quarter_name` ↔ `population.QuarLang`
- Year: `traffic_qy.year` ↔ `population.StichtagDatJahr`

Output:
A unified dataset (`traffic_pop`) containing:
- quarter_name
- quarter_id
- year
- total_traffic
- total_population


In [48]:
# Prepare population dataset for merging
population_renamed = population.rename(
    columns={
        "QuarLang": "quarter_name",
        "StichtagDatJahr": "year"
    }
)

# Merge traffic totals with population totals
traffic_pop = traffic_qy.merge(
    population_renamed[["quarter_name", "year", "total_population"]],
    on=["quarter_name", "year"],
    how="left"
)

print("Merged traffic + population dataset:", traffic_pop.shape)
traffic_pop.head(10)


KeyError: "['total_population'] not in index"

## Compute Year-over-Year Growth Rates

### Compute Year-over-Year Growth Rates

Goal:
Calculate yearly growth for both traffic volume and population in each quarter.

Method:
1. Sort the merged dataset by `quarter_id` and `year`.
2. Use `groupby` + `pct_change()` to compute the percentage change:
   - `traffic_growth_pct = pct_change(total_traffic)`
   - `population_growth_pct = pct_change(total_population)`
3. Multiply by 100 to convert to percentages.

Output:
Two new columns:
- `traffic_growth_pct`
- `population_growth_pct`

These will be used to compute the Stress Index in the next step.


In [ ]:
#  Compute traffic and population growth

# Ensure correct sorting
traffic_pop = traffic_pop.sort_values(["quarter_id", "year"])

# Compute year-over-year growth
traffic_pop["traffic_growth_pct"] = (
    traffic_pop.groupby("quarter_id")["total_traffic"].pct_change() * 100
)

traffic_pop["population_growth_pct"] = (
    traffic_pop.groupby("quarter_id")["total_population"].pct_change() * 100
)

print("Growth columns added:")
traffic_pop.head(10)


## Computing the Stress Index

###  Compute the Stress Index

The Stress Index measures whether traffic growth outpaces population growth:
$$
\text{Stress Index} = \text{traffic growth (\%)} - \text{population growth (\%)}
$$
Interpretation:
- **Positive values:** Traffic grows faster → more external inflow → *Commuter Hub*
- **Negative values:** Population grows faster → streets used more by residents → *Residential Zone*
- **Near zero:** Traffic and population change together → *Balanced*


In [ ]:
# Compute Stress Index
traffic_pop["stress_index"] = (
    traffic_pop["traffic_growth_pct"] - traffic_pop["population_growth_pct"]
)

print("Stress Index added:")
traffic_pop.head(10)


• Stress Index = Traffic Growth (%) – Population Growth (%)

• Positive Stress Index → traffic grows faster than population
  * Example: 2013 in Rathaus
    - Traffic growth: +3.59%
    - Population growth: +0.83%
    - Stress Index: +2.77% → indicates more commuter pressure

• Negative Stress Index → population grows faster or traffic drops
  * Example: 2015 in Rathaus
    - Traffic growth: –37.83%
    - Population growth: +0.82%
    - Stress Index: –38.65% → reflects a major traffic decline (construction or route changes)

• Very large positive Stress Index → strong traffic rebound or sudden inflow
  * Example: 2016 in Rathaus
    - Traffic growth: +57.58%
    - Population growth: –0.78%
    - Stress Index: +58.36% → could be heavy commuter influence or after construction work is done

• Near-zero Stress Index → traffic and population change together
   * Example: 2021 in Rathaus
    - Traffic growth: +2.52%
    - Population growth: +0.86%
    - Stress Index: +1.67% → relatively balanced

• Population growth is stable each year (around +0.6% to +1%)
• Traffic growth is highly volatile due to real-world factors:
  - roadworks
  - closures
  - sensor outages
  - seasonal variation
  - COVID impact (e.g., 2020 shows reduced traffic)

• Stress Index values in the dataset follow realistic patterns and confirm:
  - traffic does not behave like population
  - commuter flows strongly affect certain quarters
  - volatility in traffic explains extreme stress values


## Final Dataset Export for Tableau

In [ ]:
traffic_pop_export = traffic_pop.rename(columns={
    "quarter_name": "quarter",
    "quarter_id": "quarter_id",
    "year": "year",
    "total_traffic": "total_traffic",
    "total_population": "total_population",
    "traffic_growth_pct": "traffic_growth_pct",
    "population_growth_pct": "population_growth_pct",
    "stress_index": "stress_index",
    "category": "category"
})


In [ ]:
export_path = BASE + "stress_index_tableau_ready.csv"

traffic_pop_export.to_csv(export_path, index=False)

print("Exported Tableau dataset to:", export_path)


In [ ]:
df_export = pd.read_csv(BASE + "stress_index_tableau_ready.csv")
df_export.head(20)


### Classify Quarters Based on Stress Index (Optional)

To make the Stress Index easier to interpret and visualize, we classify each quarter–year
combination into three categories:

- Commuter Hub: Stress Index > +5  
- Residential Zone: Stress Index < –5  
- Balanced: Stress Index between –5 and +5  

This classification helps identify which quarters experience mobility pressure from commuters,
which are more residential, and which remain stable. This enhances storytelling and supports
clear thematic mapping in Tableau.

In [ ]:
# Create a classification based on Stress Index

def classify_stress(x):
    if x > 5:
        return "Commuter Hub"
    elif x < -5:
        return "Residential Zone"
    else:
        return "Balanced"

df_export["category"] = df_export["stress_index"].apply(classify_stress)

print("Classification added:")
df_export.head(20)


**Classification shows which quarters are commuter hubs, residential zones, or balanced.**
* Commuter Hubs: quarters where stress index exceeds +5, indicating traffic growth far above population growth.
* Residential Zones: stress index below –5, where population grows faster or traffic drops.
* Balanced Quarters: stress index between –5 and +5, indicating stable mobility patterns.
* This classification improves the thematic map and helps identify pressure points in the city's mobility network.


# Create a new export dataset including the classification column
traffic_pop_export_classes = traffic_pop.rename(columns={
    "quarter_name": "quarter",
    "quarter_id": "quarter_id",
    "year": "year",
    "total_traffic": "total_traffic",
    "total_population": "total_population",
    "traffic_growth_pct": "traffic_growth_pct",
    "population_growth_pct": "population_growth_pct",
    "stress_index": "stress_index",
    "category": "category"
})

# New file path
export_path_classes = BASE + "stress_index_tableau_with_classes.csv"

# Save to disk
traffic_pop_export_classes.to_csv(export_path_classes, index=False)

print("Exported classified Tableau dataset to:", export_path_classes)
